# 전처리 과정

- RMSE : 26185.4795
- 데이터 : train_csv, test_csv

In [ ]:
# 데이터 로드 후
# 전처리 위한 데이터셋 합치기

train['data'] = 0
test['data'] = 1
concat = pd.concat([train, test])

In [ ]:
# 이름 바꾸기

concat = concat.rename(columns={'전용면적(㎡)':'전용면적'})

In [ ]:
# 본번, 부번의 경우 float로 되어있지만 범주형 변수의 의미를 가지므로 object(string) 형태로 바꾸기
concat_select['본번'] = concat_select['본번'].astype('str')
concat_select['부번'] = concat_select['부번'].astype('str')

### 결측치 탐색 및 처리

In [ ]:
# 결측치 탐색 후 결측률 보기
# 이후에 의미 없는 값들 찾아서 채우기
# 위 처럼 아무 의미도 갖지 않는 칼럼은 결측치와 같은 역할을 하므로, np.nan으로 채워 결측치로 인식시킴

concat['도로명'] = concat['도로명'].replace(' ', np.nan)
concat['등기신청일자'] = concat['등기신청일자'].replace(' ', np.nan)
concat['거래유형'] = concat['거래유형'].replace('-', np.nan)
concat['중개사소재지'] = concat['중개사소재지'].replace('-', np.nan)
concat['k-시행사'] = concat['k-시행사'].replace('.', np.nan)
concat['k-시행사'] = concat['k-시행사'].replace('-', np.nan)
concat['k-홈페이지'] = concat['k-홈페이지'].replace('없음', np.nan)
concat['k-홈페이지'] = concat['k-홈페이지'].replace('.', np.nan)

In [ ]:
# print(concat.shape[0] * 0.8) = 902475.2000000001
# 결측값이 80%이상인(Null값이 90만개 이상) 칼럼은 삭제
print('* 결측치가 90만개 이하인 변수들 :', list(concat.columns[concat.isnull().sum() <= 900000]))     # 남겨질 변수들은 아래와 같습니다.
print('* 결측치가 90만개 이상인 변수들 :', list(concat.columns[concat.isnull().sum() >= 900000]))

# 결측치 90만개 이상인 값과 이하지만 필요없는 것 제외
# 필요없어 보이는 것 : k-전화번호, k-팩스번호, 사용허가여부, 관리비 업로드, k-수정일자
valid_cols = concat.columns[concat.isnull().sum() <= 900000]
exclude_cols = ['k-전화번호', 'k-팩스번호', '사용허가여부', '관리비 업로드', 'k-수정일자']

select = [col for col in valid_cols if col not in exclude_cols]
concat_select = concat[select]

concat.shape, concat_select.shape

In [ ]:
# 먼저, 연속형 변수와 범주형 변수를 위 info에 따라 분리
# 숫자형 분리 pd.api.types.is_numeric_dtype
con_columns = []
cat_columns = []

for column in concat_select.columns:
    if pd.api.types.is_numeric_dtype(concat_select[column]):
        con_columns.append(column)
    else:
        cat_columns.append(column)

In [ ]:
#상관관계 기반으로 중복 feature 쌍 탐지 및 삭제 후보 추천

# 연속형 변수만 추출
numeric_cols = con_columns  # ← 너가 나눠둔 연속형 변수 리스트

# 상관관계 행렬 (절댓값 기준)
corr_matrix = concat_select[numeric_cols].corr().abs()

# 상삼각 행렬로 중복 제거
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# 상관계수 0.9 초과인 변수쌍 추출
high_corr_pairs = [(col, row, upper.loc[row, col])
                   for col in upper.columns
                   for row in upper.index
                   if pd.notnull(upper.loc[row, col]) and upper.loc[row, col] > 0.7]

# 출력
for col1, col2, score in sorted(high_corr_pairs, key=lambda x: -x[2]):
    print(f"🔁 {col1} ↔ {col2} : 상관계수 = {score:.2f}")


In [ ]:
# 전용면적별세대현황 pca

pca_cols = [
    'k-전용면적별세대현황(60㎡이하)',
    'k-전용면적별세대현황(60㎡~85㎡이하)',
    'k-85㎡~135㎡이하']
pca_data = concat_select[pca_cols].fillna(0)  # 혹시 모르니 결측 0으로 대체

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_pca_data = scaler.fit_transform(pca_data)

# 2개 성분으로 축소
from sklearn.decomposition import PCA

pca = PCA(n_components=2)  
pca_components = pca.fit_transform(scaled_pca_data)

# PCA 설명력 보기
print(pca.explained_variance_ratio_)  # 예: [0.83, 0.16]

# PCA 결과 저장
concat_select["세대면적_PCA1"] = pca_components[:, 0]
concat_select["세대면적_PCA2"] = pca_components[:, 1]

# 원본 feature 제거
concat_select.drop(columns=pca_cols, inplace=True)

In [ ]:
# 연속형 변수 상관관계 기반으로 중복의 의미를 갖는 값 삭제

drop_cols = ['k-관리비부과면적','k-연면적','k-전체동수']
concat_select.drop(columns=drop_cols, inplace=True)

In [ ]:
# 연속형 변수 동 단위 평균으로 결측치 채우기
# 동이 없으면 구, 구가 없으면 전체 평균으로 채우기
# target은 건들지 말아보자

concat_select['구'] = concat_select['시군구'].str.split().str[1]
concat_select['동'] = concat_select['시군구'].str.split().str[2]

impute_targets = ['건축면적', '주차대수', '좌표X', '좌표Y', 'k-주거전용면적', 'k-전체세대수']

for col in impute_targets:
    # 1차: 동 단위 평균
    concat_select[col] = concat_select.groupby('동')[col].transform(lambda x: x.fillna(x.mean()))
    # 2차: 구 단위 평균 (동 평균이 안 되면 여기서)
    concat_select[col] = concat_select.groupby('구')[col].transform(lambda x: x.fillna(x.mean()))
    # 3차: 전체 평균 (구 평균도 안 되면 여기서)
    concat_select[col].fillna(concat_select[col].mean(), inplace=True)


In [ ]:
# 범주형 feature들 관계 보기

from scipy.stats import chi2_contingency

# Cramér's V 계산 함수
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix, correction=False)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    return np.sqrt(phi2 / min(k - 1, r - 1))

# 범주형 변수 리스트
cat_cols = cat_columns2  # 이미 나눈 리스트

In [ ]:
# 크래머스 브이 기준 관계있는 범주형 변수 삭제
drop_cat_cols = [
    '본번',
    '부번',
    '도로명',
    '단지승인일',
    '단지신청일',
    'k-세대타입(분양형태)',
    'k-관리방식',
    'k-난방방식',
    'k-복도유형',
    '세대전기계약방법',
    '경비비관리형태',
    '청소비관리형태',
    '기타/의무/임대/임의=1/2/3/4',
]

# 제거 적용
concat_select.drop(columns=drop_cat_cols, inplace=True)

### 이상치 탐지 및 처리

In [ ]:
# IQR로 이상치 탐지
def detect_outliers_iqr(df, columns, iqr_scale=1.5):
    outlier_summary = []

    for col in columns:
        if df[col].isnull().all():
            continue

        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - iqr_scale * IQR
        upper_bound = Q3 + iqr_scale * IQR

        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outlier_count = outliers.shape[0]
        outlier_ratio = outlier_count / df.shape[0] * 100

        outlier_summary.append({
            '변수': col,
            '이상치 개수': outlier_count,
            '이상치 비율(%)': round(outlier_ratio, 2)
        })

    return pd.DataFrame(outlier_summary).sort_values('이상치 비율(%)', ascending=False)


In [ ]:
# pca 설명력이 높지도 않고 이상치도 많아서 pca 제거
# 원본 k-전용면적별세대현황(60㎡~85㎡이하) 가져오기

# 1. PCA로 만든 feature 제거
pca_cols = ['세대면적_PCA1', '세대면적_PCA2']
concat_select.drop(columns=pca_cols, inplace=True, errors='ignore')

# 2. 원본에서 특정 변수만 가져와서 추가
selected_feature = 'k-전용면적별세대현황(60㎡~85㎡이하)'
concat_select[selected_feature] = concat[selected_feature]


In [ ]:
# concat_select가 아닌 concat에서 가져와서 결측치 확인

concat_select['k-전용면적별세대현황(60㎡~85㎡이하)'].isnull().sum()

In [ ]:
# 결측치는 동/구/전체 평균으로 채우기

impute2_targets = ['k-전용면적별세대현황(60㎡~85㎡이하)']

for col in impute2_targets:
    # 1차: 동 단위 평균
    concat_select[col] = concat_select.groupby('동')[col].transform(lambda x: x.fillna(x.mean()))
    # 2차: 구 단위 평균 (동 평균이 안 되면 여기서)
    concat_select[col] = concat_select.groupby('구')[col].transform(lambda x: x.fillna(x.mean()))
    # 3차: 전체 평균 (구 평균도 안 되면 여기서)
    concat_select[col].fillna(concat_select[col].mean(), inplace=True)

In [ ]:
# 결측치 적용 및 다음 이상치 탐지

# 리스트 초기화
con_columns_final = []
cat_columns_final = []

# concat_select 기준으로 분리
for col in concat_select.columns:
    if pd.api.types.is_numeric_dtype(concat_select[col]):
        con_columns_final.append(col)
    else:
        cat_columns_final.append(col)

def detect_outliers_iqr(df, columns, iqr_scale=1.5):
    outlier_summary = []

    for col in columns:
        if df[col].isnull().all():
            continue

        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - iqr_scale * IQR
        upper_bound = Q3 + iqr_scale * IQR

        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outlier_count = outliers.shape[0]
        outlier_ratio = outlier_count / df.shape[0] * 100

        outlier_summary.append({
            '변수': col,
            '이상치 개수': outlier_count,
            '이상치 비율(%)': round(outlier_ratio, 2)
        })

    return pd.DataFrame(outlier_summary).sort_values('이상치 비율(%)', ascending=False)


In [ ]:
# 전용세대별면적PCA 변수 외 이상치 발견값 clip

def clip_iqr(df, columns, k=1.5):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - k * IQR
        upper = Q3 + k * IQR
        df[col] = df[col].clip(lower, upper)
    return df

clip_cols = ['건축면적', '전용면적', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-전체세대수', '주차대수']

concat_select = clip_iqr(concat_select, clip_cols)

### 모델 학습을 위한 처리

In [ ]:
# 이제 다시 train과 test dataset을 분할해줍니다. 위에서 제작해 놓았던 is_test 칼럼을 이용합니다.
dt_train = concat_select.query('data==0')
dt_test = concat_select.query('data==1')

# 이제 is_test 칼럼은 drop해줍니다.
dt_train.drop(['data'], axis = 1, inplace=True)
dt_test.drop(['data'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)

In [ ]:
# 범주형 변수 인코딩
# 파생변수 제작으로 추가된 변수들이 존재하기에, 다시한번 연속형과 범주형 칼럼을 분리해주겠습니다.
continuous_columns_final = []
categorical_columns_final = []

for column in dt_train.columns:
    if pd.api.types.is_numeric_dtype(dt_train[column]):
        continuous_columns_final.append(column)
    else:
        categorical_columns_final.append(column)

print("연속형 변수:", continuous_columns_final)
print("범주형 변수:", categorical_columns_final)

In [ ]:
# 아래에서 범주형 변수들을 대상으로 레이블인코딩을 진행해 주겠습니다.

# 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
label_encoders = {}

# Implement Label Encoding
for col in tqdm( categorical_columns_final ):
    lbl = LabelEncoder()

    # Label-Encoding을 fit
    lbl.fit( dt_train[col].astype(str) )
    dt_train[col] = lbl.transform(dt_train[col].astype(str))
    label_encoders[col] = lbl           # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.

    # Test 데이터에만 존재하는 새로 출현한 데이터를 신규 클래스로 추가해줍니다.
    for label in np.unique(dt_test[col]):
      if label not in lbl.classes_: # unseen label 데이터인 경우
        lbl.classes_ = np.append(lbl.classes_, label) # 미처리 시 ValueError발생하니 주의하세요!

    dt_test[col] = lbl.transform(dt_test[col].astype(str))

In [ ]:
# Target과 독립변수들을 분리해줍니다.
y_train = dt_train['target']
X_train = dt_train.drop(['target'], axis=1)

# Hold out split을 사용해 학습 데이터와 검증 데이터를 8:2 비율로 나누겠습니다.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2023)